In [2]:
from __future__ import print_function
from six.moves import range
from PIL import Image
import sys

dir_path = '/home/avoyd/GANtor-Arts-Center/src/code/main.py'
sys.path.append(dir_path)
sys.path.append('/home/avoyd/GANtor-Arts-Center/src/code/')

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.optim as optim
import os
import time

import numpy as np
import torchfile

from miscc.config import cfg, cfg_from_file
from miscc.utils import mkdir_p
from miscc.utils import weights_init
from miscc.utils import save_img_results, save_model
from miscc.utils import KL_loss
from miscc.utils import compute_discriminator_loss, compute_generator_loss

from tensorboard import summary
from tensorboardX import FileWriter

import torchvision
import torchvision.utils as vutils
from matplotlib import pyplot as plt

import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

import torchvision.transforms as transforms

import datetime
import dateutil
import dateutil.tz
from PIL import Image
import csv
import matplotlib.pyplot as plt
import numpy as np



from miscc.datasets import TextDataset
from miscc.utils import mkdir_p
from trainer import GANTrainer

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device(0)
print(device)
print(torch.cuda.is_available())

cuda:0
True


In [6]:
def initialize_inception_wikiart(load_path, num_classes, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.

    model_ft = torchvision.models.inception_v3(pretrained=use_pretrained)
    #set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)

    if load_path != None:
        model_ft.load_state_dict(torch.load(load_path))
        
    return model_ft



In [7]:
def inception_score(inception_model, imgs, num_classes=27, cuda=True, batch_size=32, resize=False, splits=1,
                    gpu_list=None):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size, num_workers=8, drop_last=True)

#     # Load inception model
#     inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
#     inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        if gpu_list != None:
            #Parallelize across >1 GPUs
            x = nn.parallel.data_parallel(inception_model, x, GPU_list)
        else:
            x = inception_model(x)
            
        return F.softmax(x, dim=1).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, num_classes))

    for i, batch in enumerate(dataloader, 0):
#         batch = torch.tensor(batch)
        batch = batch[0].type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

if __name__ == '__main__':

    # Specify classification category and image size
    category = "genre"
    num_classes = 27 if category == "style" else 10
    
    image_size = 64 # 64 for S1 outs, 256 for S2 outs
    
    gen_sample_dir = './baseline_generated/{}{}/'.format(category, image_size) 
    batch_size = 32
    
    #Trained for 5 eps on style
#     inception_path = './ft_wikiart/ft_{}_5eps.pth'.format(category)

    #Trained for 15 eps on genre
    inception_path = './ft_wikiart/ft_style_5eps.pth' if category == "style" else './ft_wikiart/ft_genre_15eps.pth'
    
    print("Loading inception v3 weights from {}".format(inception_path))
#     image_transform = transforms.Compose(
#         [transforms.CenterCrop(GAN_inp_size),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
    image_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    gen_sample_dataset = torchvision.datasets.ImageFolder(gen_sample_dir, transform=image_transform)

    #Model expects input size of 299
    inception_model = initialize_inception_wikiart(inception_path, num_classes)
    inception_model.to(device)
    inception_model.eval()
    inception_model = nn.DataParallel(inception_model)
    
    print ("Calculating Inception Score...")
    score_mean, score_std = inception_score(inception_model, gen_sample_dataset, 
                                      num_classes=num_classes, cuda=True, batch_size=batch_size, resize=True, splits=10)
    
    print("Inception score, mean and std: {} +- {}".format(score_mean, score_std))

Loading inception v3 weights from ./ft_wikiart/ft_genre_15eps.pth
Calculating Inception Score...


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Inception score, mean and std: 1.3359176652675087 +- 0.0777914503390635


In [10]:
############IGNORE################################

#     class IgnoreLabelDataset(torch.utils.data.Dataset):
#         def __init__(self, orig):
#             self.orig = orig

#         def __getitem__(self, index):
#             return self.orig[index][0]

#         def __len__(self):
#             return len(self.orig)

# #     cifar = dset.CIFAR10(root='data/', download=True,
# #                              transform=transforms.Compose([
# #                                  transforms.Scale(32),
# #                                  transforms.ToTensor(),
# #                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# #                              ])
#     )
print(score_mean, score_std)

1.6372140809617561 0.22283080726758986


In [ ]:
## Unused func
# def initialize_Generator(
#     config_path='../cfg/wikiart_s2.yml', 
#     S1_path = '../../../results/wikiart_stageI_2019_05_14_23_03_43/Model/netG_epoch_60.pth',
#     S2_path = '../../../results/wikiart_stageII_2019_05_15_03_54_54/Model/netG_epoch_45.pth'
# ):
    
#     cfg_from_file(config_path)

#     from model import STAGE1_G, STAGE2_G, STAGE2_D

#     Stage1_G = STAGE1_G()

#     netG = STAGE2_G(Stage1_G)
#     netG.apply(weights_init)

#     stage_1_file = S1_path
#     stage_2_file = S2_path

#     state_dict = torch.load(stage_2_file, map_location=lambda storage, loc: storage)
#     netG.load_state_dict(state_dict)
#     print('Load from: ', stage_2_file)

#     state_dict = torch.load(stage_1_file,map_location=lambda storage, loc: storage)
#     netG.STAGE1_G.load_state_dict(state_dict)
#     print('Load from: ', stage_1_file)

#     if cfg.CUDA:
#         netG.cuda()

#     print(netG)
    
#     return netG